### System matrices

In [1]:
import numpy as np

mass = np.array(
    [
        [9, 0],
        [0, 1]
    ],
    dtype=np.float64
)

damping = np.array(
    [
        [2.7, -0.3],
        [-0.3, 0.3]
    ],
    dtype=np.float64
)

stiffness = np.array(
    [
        [27, -3],
        [-3, 3]
    ],
    dtype=np.float64
)

### Modal decoupling

In [2]:
from pybeam.modal_parameters import get_modal_parameters
from pybeam._utilities import pprint_array

eigenfrequencies, modeshapes = get_modal_parameters(stiffness, mass, normalize=True)

modal_mass = modeshapes.T.dot(mass).dot(modeshapes)
modal_stiffness = modeshapes.T.dot(stiffness).dot(modeshapes)
modal_damping = modeshapes.T.dot(damping).dot(modeshapes)
damping_ratios = np.diag(modal_damping) / (2 * eigenfrequencies)


print("Modal mass")
pprint_array(modal_mass)
print("")
print("Eigenfrequencies from eigenvalue problem")
pprint_array(np.diag(eigenfrequencies))
print("")
print("Eigenfrequencies from modal decoupling")
pprint_array(np.sqrt(modal_stiffness))
print("")
print("Damping ratios from modal decoupling")
pprint_array(np.diag(damping_ratios))


Modal mass
[[1. 0.]
 [0. 1.]]

Eigenfrequencies from eigenvalue problem
[[1.41421356 0.        ]
 [0.         2.        ]]

Eigenfrequencies from modal decoupling
[[1.41421356 0.00000003]
 [0.00000002 2.        ]]

Damping ratios from modal decoupling
[[0.07071068 0.        ]
 [0.         0.1       ]]


### Time domain simulation

In [15]:
import numpy as np
from pybeam.simulation import simulate

# Number of DOF
n_dof = stiffness.shape[0]

# Time step size
dt = 0.0001

# End time
t_end = 100
# Time steps
time = np.arange(start=0, stop=t_end + dt, step=dt)

# Loading
loads = np.zeros((n_dof, len(time)))

# Initial conditions
x_0 = np.zeros((n_dof,))
x_0[0] = 10.25
x_0[1] = -10.4
v_0 = np.zeros((n_dof,))
# v_0[0] = -1

# Integration parameter
beta = 1 / 6

x, v, a = simulate(
    stiffness=stiffness,
    mass=mass,
    damping=damping,
    initial_disp=x_0,
    initial_vel=v_0,
    loads=loads,
    time=time,
    beta=beta,
    gamma=1 / 2,  # Don't change this unless you know what you're doing
)

[pybeam.simulation] [INFO] Simulating with linear acceleration
[pybeam.simulation] [INFO] Simulating 2 DOF system.
[pybeam.simulation] [INFO] Simulating from time 0.0 to 100.0 and time step size 0.0001 (1000001 samples).


### Time response plot

In [13]:
import matplotlib.pyplot as plt
from pybeam.plotting import set_style
set_style()
%matplotlib qt5

plt.figure()
ax = plt.gca()

# Adjust the look of the plot
ax.set_xlabel("Time")
ax.set_ylabel("Displacement")
ax.set_xlim(0, time[-1])

ax.plot(time, x[0, :], color="white", linestyle="-")
ax.plot(time, x[1, :], color="red", linestyle="-")
plt.show()

In [16]:
from pybeam.plotting import plot_fft, set_style
import matplotlib.pyplot as plt
set_style()
%matplotlib qt5

plot_fft(x[0, :], time, abs_vals=True)

ax = plt.gca()

# Plot eigenfrequency indicators
for freq in eigenfrequencies:
    freq_in_hz = freq / (2 * np.pi)
    ax.axline((freq_in_hz, 0), ((freq_in_hz, 1)), linestyle="--", color="red")

# ticks = list(plt.xticks()[0]) + list(eigenfrequencies / (2 * np.pi))
ticks = list(eigenfrequencies / (2 * np.pi))
# ax.set_xscale("linear")
ax.set_xticks(ticks, [f"{val}" for val in ticks])


### Phase angle plot

In [26]:
fft = np.fft.fft(x[0, :])
phase = np.rad2deg(np.angle(fft))
n_2 = int(np.floor(len(time) / 2))
freqs = np.fft.fftfreq(len(time), dt)[0:n_2]

plt.figure()
ax = plt.gca()
ax.plot(freqs, phase[0:n_2])
ax.set_xlabel("Frequency")
ax.set_ylabel("Angle")
ax.set_xscale("log")

# Plot eigenfrequency indicators
for freq in eigenfrequencies:
    freq_in_hz = freq / (2 * np.pi)
    ax.axline((freq_in_hz, 0), ((freq_in_hz, 1)), linestyle="--", color="red")
